In [1]:
import numpy as np
import requests
import time
from tensorflow.keras.optimizers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation

from pyspark import SQLContext, SparkContext
from pyspark import SparkConf

from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.linalg import Vectors


from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold

Using TensorFlow backend.


In [2]:
# Load the dataset and labels
x=np.load('MCx.npy')
y=np.load('MCy.npy')

In [3]:
conf = SparkConf()
sc = SparkContext(conf = conf)
spark = SQLContext(sc)

In [4]:
# Calculate the rank of each feature
R=[]
for h in range(x.shape[1]):
    kmeans = KMeans(init='k-means++', n_clusters=np.unique(y).shape[0], n_init=10)
    ff=kmeans.fit_predict(x[:,h].reshape(-1,1))
    r=metrics.homogeneity_score(y,ff) #Use the homogeneity score as a rank of the feature
    R.append(r)

In [5]:
#Arrange feature accroding to thier ranks
Rnk=np.argsort(np.array(R))

In [6]:
#Initiate the cross-validation splitter
kfolds=StratifiedKFold(n_splits=5,shuffle=True)

In [7]:
#Per each set of ranks, use cross-validation to calculate accuracy.
smr=[]
for j in range(Rnk.shape[0]):
    fd=x[:,Rnk[j:]]
    pp=0
    lpa=np.zeros((0,2))
    for train,test in kfolds.split(fd,y):
        dff = map(lambda x: (int(float(x[-1])), Vectors.dense(x[:-1])),np.hstack((fd[train],y[train].reshape(-1,1))))
        TrD = spark.createDataFrame(dff,schema=["label", "features"])
        dff = map(lambda x: (int(float(x[-1])), Vectors.dense(x[:-1])),np.hstack((fd[test],y[test].reshape(-1,1))))
        TsD = spark.createDataFrame(dff,schema=["label", "features"])
        model = Sequential()
        model.add(Dense(128,input_dim=fd.shape[1],activation='relu',use_bias=True))
        model.add(Dropout(0.5))
        model.add(Dense(64,activation='relu',use_bias=True))
        model.add(Dropout(0.5))
        model.add(Dense(32,activation='relu',use_bias=True))
        model.add(Dropout(0.5))
        model.add(Dense(np.unique(y).shape[0],activation='softmax',use_bias=True)) #The number of neurons is equal to the number of classes
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
        trainer = SingleTrainer(keras_model=model, worker_optimizer='adam', loss='categorical_crossentropy', num_epoch=1000)
        trained_model = trainer.train(TrD)
        st = time.time()
        predictor = ModelPredictor(keras_model=trained_model)
        et=time.time()-st
        ff=predictor.predict(TsD)
        ts=np.array(map(lambda x: x[0],ff.select('prediction').collect())).reshape(-1,1)
        pp=pp+metrics.accuracy_score(y[test].reshape(-1,1),(ts>=0.5).astype(int))
        lpa=np.vstack((lpa,np.hstack((y[test].reshape(-1,1),ts))))
    pp=pp/kfolds.n_splits
    np.savetxt('F%d.csv'%j,lpa,delimiter=',')
    smr.append([j, pp, et*1000000/x.shape[0]]) #Calculate the time required to predict a label per each object in uS.

In [8]:
smr

[[43.0, 0.970176, 5.155115583],
 [42.0, 0.969858, 5.057759047],
 [41.0, 0.970432, 4.960576812],
 [40.0, 0.969808, 4.863301202],
 [39.0, 0.970264, 4.814621377],
 [38.0, 0.970027, 5.680481071],
 [37.0, 0.970458, 4.620123069],
 [36.0, 0.970387, 4.66872508],
 [35.0, 0.969518, 4.425599861],
 [34.0, 0.970013, 4.668787331],
 [33.0, 0.969926, 4.328346038],
 [32.0, 0.969854, 4.182393715],
 [31.0, 0.969613, 4.08511188],
 [30.0, 0.970438, 3.987917194],
 [29.0, 0.970464, 4.710921524],
 [28.0, 0.970041, 3.793384648],
 [27.0, 0.970099, 3.744754624],
 [26.0, 0.970192, 3.744742174],
 [25.0, 0.970076, 3.696102813],
 [24.0, 0.969492, 3.550110028],
 [23.0, 0.968671, 3.433941416],
 [22.0, 0.969088, 3.337039308],
 [21.0, 0.969155, 3.277643075],
 [20.0, 0.969102, 3.584238817],
 [19.0, 0.968871, 3.122605304],
 [18.0, 0.968506, 3.015145526],
 [17.0, 0.969479, 3.015157976],
 [16.0, 0.969028, 2.820715693],
 [15.0, 0.968624, 2.772110569],
 [14.0, 0.968287, 2.674757146],
 [13.0, 0.967739, 2.67470112],
 [12.0, 0.9